In [87]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sci
import numpy as np
import numpy.linalg as linalg
import numpy.random as rand
import pandas as pd
from platform import node
from cdlib.classes.node_clustering import NodeClustering
import community.community_louvain as community_louvain
import cdlib as cd
from collections import defaultdict
from collections import Counter
import networkx as nx
import cdlib.algorithms as algo
import cdlib.viz as viz
import matplotlib.pyplot as plt

from functions import louvain

In [88]:
def get_degrees(edges):
    counter = Counter(edges)
    return counter.most_common()

In [89]:
# reading in the data, creating the graph

hero_edges = pd.read_csv('hero-network.csv')
nodes = pd.read_csv('nodes.csv')
edges = pd.read_csv('edges.csv')

hero_graph = nx.from_pandas_edgelist(hero_edges, 'hero1', 'hero2', create_using=nx.Graph())

In [90]:
# get degrees of each node

hero_nodes = []
for x in range(len(nodes['node'])):
    if nodes['type'][x] != 'comic':
        hero_nodes.append(nodes['node'][x])

stubs1 = list(hero_edges['hero1'])
stubs2 = list(hero_edges['hero2'])

stubs = stubs1 + stubs2

hero_degrees = get_degrees(stubs)
degrees = [x[1] for x in hero_degrees]
heroes_sorted = [x[0] for x in hero_degrees]

In [93]:
MarvelCluster = louvain(hero_graph)

In [95]:
print(len(MarvelCluster.communities))
sort_coms = []
for i in range(len(MarvelCluster.communities)):
    MarvelCluster.communities[i] = sorted(MarvelCluster.communities[i], key = heroes_sorted.index)

25


In [97]:
coms = pd.DataFrame(MarvelCluster.communities)
coms = coms.transpose()



In [98]:
coms.head(20)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,WOLVERINE/LOGAN,SPIDER-MAN/PETER PAR,CAPTAIN AMERICA,SHE-HULK/JENNIFER WA,THING/BENJAMIN J. GR,HULK/DR. ROBERT BRUC,THOR/DR. DONALD BLAK,BEETLE/ABNER RONALD,DR. STRANGE/STEPHEN,"CAGE, LUKE/CARL LUCA",...,QABIRI,"LUDLUM, ROSS",CHI-CHEE,KARMA 2013,THINGEE,PANTHER CUB/,DR. MIDAS/,"BAPTISTE, TANTE MATT","KOLE, MICHELLE",STEEL SPIDER/OLLIE O
1,BEAST/HENRY &HANK& P,"JAMESON, J. JONAH",IRON MAN/TONY STARK,HERCULES [GREEK GOD],HUMAN TORCH/JOHNNY S,"JONES, RICHARD MILHO",ODIN [ASGARDIAN],MOONSTONE II/KARLA S,WONG,IRON FIST/DANIEL RAN,...,"ZEITGEIST, NICOLA",ORWELL,HOPE,TANGERINE 2013,"DOCTOR, DR. JOSHUA",STERLING,MARVEL BOY VIII/NOH-,SISTER KATRINA,"HIGGINS, JANE",MASTER OF VENGEANCE
2,CYCLOPS/SCOTT SUMMER,"WATSON-PARKER, MARY",SCARLET WITCH/WANDA,BLACK KNIGHT V/DANE,MR. FANTASTIC/REED R,SASQUATCH/WALTER LAN,BALDER [ASGARDIAN],POWER MAN/ERIK JOSTE,CLEA,"KNIGHT, MISTY",...,CITYDWELLER/JERRY |,"ASHER, MICHAEL",MAINSPRING,PSYLOCKE 2013,"BENNY, JAKE",MISS THING/MARY,PLEX INTELLIGENCE,SYBIL,"HIGGINS, RICHIE",None
3,STORM/ORORO MUNROE S,DAREDEVIL/MATT MURDO,VISION,QUASAR III/WENDELL V,INVISIBLE WOMAN/SUE,"HUDSON, HEATHER",HOGUN [ASGARDIAN],SCREAMING MIMI/MELIS,GHOST RIDER II/JOHNN,"WING, COLLEEN",...,THOR | EARTH 253,FAGIN,PSIREN,DOUGLOCK 2013,"FILLMORE, GARRET",AMAZO-MAXI-WOMAN/,CAPTAIN GLORY,ANIELLE,"KOLE, JOHN",None
4,COLOSSUS II/PETER RA,"ROBERTSON, JOE",HAWK,NOVA/RICHARD RIDER,SUB-MARINER/NAMOR MA,"NORRISS, SISTER BARB",FANDRAL [ASGARDIAN],NIGHT THRASHER/DUANE,GHOST RIDER III/DAN,SHANG-CHI,...,PROFESSOR X | EARTH,HOFFMAN,PSIMON,WOLFSBANE 2013,"FILLMORE, BETTY","DARLEGUNG, GEN.",MERREE,"STOKER, OLIVIER","HIGGINS, LINDA",None
5,PROFESSOR X/CHARLES,"PARKER, MAY",WASP/JANET VAN DYNE,SILVER SURFER/NORRIN,PATRIOT/JEFF MACE,PUCK/EUGENE MILTON J,SIF,CITIZEN V II/HELMUT,RINTRAH,BOVA,...,SISTER PERPETUA,OSWALD,SCENE STEALER,MEGGAN 2013,NUTT,SWORDSMAN IV/,OUBLIETTE/,None,None,None
6,MARVEL GIRL/JEAN GRE,"LEEDS, BETTY BRANT",ANT-MAN/DR. HENRY J.,DR. DOOM/VICTOR VON,CRYSTAL [INHUMAN],SHAMAN/MICHAEL TWOYO,VOLSTAGG,FIXER II/PAUL NORBER,DORMAMMU,"TARR, BLACK JACK",...,TECHNOCRAT/RANDY,"ASHER, CARL",FIELDER,"WISDOM, PETE 2013","LILLIAN, BEA",MANT/ERNEST,None,None,None,None
7,ANGEL/WARREN KENNETH,"THOMPSON, EUGENE FLA",WONDER MAN/SIMON WIL,FIRESTAR/ANGELICA JO,MISS AMERICA/MADELIN,NORTHSTAR/JEAN-PAUL,LOKI [ASGARDIAN],JOLT/HALLIE TAKAHAMA,"WOLFE, SARA","WU, LEIKO",...,WHITE BIRD/AMARA,"ASHER, DONNA",CHECK,CAPTAIN BRITAIN 2013,"GALE, SHERIFF",None,None,None,None,None
8,ICEMAN/ROBERT BOBBY,KINGPIN/WILSON FISK,QUICKSILVER/PIETRO M,MOONDRAGON/HEATHER D,"RICHARDS, FRANKLIN B","BANNER, BETTY ROSS T",SERSI/SYLVIA,KLAW/ULYSSES KLAW,NIGHTMARE/EDVARD HAB,"RESTON, CLIVE",...,NIGHTFIGHTER/,"NILES, SEN. CATHERIN",RELAY,None,None,None,None,None,None,None
9,NIGHTCRAWLER/KURT WA,"NELSON, FRANKLIN FOG","JARVIS, EDWIN",NAMORITA/NITA PRENTI,MEDUSA/MEDUSALITH AM,AURORA/JEANNE-MARIE,ENCHANTRESS/AMORA/HE,CHARCOAL/CHARLIE BUR,BARON MORDO/KARL MOR,"SMITH, SIR DENIS NAY",...,IDRIS,None,None,None,None,None,None,None,None,None


In [99]:
spec_cluster = algo.r_spectral_clustering(hero_graph,  n_clusters=25, method="vanilla")

In [103]:
spec_coms = pd.DataFrame(spec_cluster.communities).transpose()
spec_coms.head(20)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,"LITTLE, ABNER","ERWIN, CLYTEMNESTRA",PYRO/ALLERDYCE JOHNN,ETERNITY/ADAM QADMON,LIVING LASER/ARTHUR,DEATH,GOG III,"ZIMMER, ABE","JACKSON, STEVE","STARR, TRIXIE TRISH",...,CLOUD,GREEN GOBLIN III/BAR,WONG,"KRIEGHUND, COL.",SHOCKER/HERMAN SCHUL,ASANO,DOMINUS,"KAFKA, DR. ASHLEY",KARMA/XI'AN COY MANH,"TWAKI, DR. TED"
1,PRINCESS ZANDA,IRON MAN IV/JAMES R.,"JONES, LORRAINE LORR",KARNAK [INHUMAN],"CLEMSON, MIKE","BUCKNER, GRANT",KRAKOA,"JONES, DANIEL DANNY",GHOST RIDER III/DAN,"PROUST, PAUL",...,BLOODSTONE/ULYSSES B,SLASHER,"MAJCOMB, RENEE","DELGADO, FATHER","WEIN, LEN",SICKLE,"MARRS(-PAYNE), PHOEB",SCORCHER/,MASTER/ESHU,SISTER MARIA
2,BLACK PANTHER/T'CHAL,"TARR, BLACK JACK",CAPTAIN AMERICA,ULTRON,"GARTH, DONNA",GORGILLA,"KORVAC, MICHAEL","BYRNES, GAYLE WATSON",VOLSTAGG,SILVER DAGGER/ISAIAH,...,"FURY, COL. NICHOLAS",MAX,"TENG, NIKKI","RAHN, TAMARA",IRON FIST H'YLTHRI I,"DAVIS, PRINCIPAL","DILLON, JONATHAN","PEEL, EMMA",RICOCHET/SEN. STEVEN,TANGERINE 2013
3,"STEELE, SIMON/WOLFGA","SMITH, SIR DENIS NAY",ANDROMEDA/ANDROMEDA,DAZZLER II/ALLISON B,MOLE MAN/HARVEY RUPE,"VAUGHAN, MRS.",LEADER/SAM STERNS,"MANNING, DR.","IONELLO, JASON",HOBGOBLIN II/RODERIC,...,STRONG GUY/GUIDO CAR,"MICHEL, RAFE",LUMP,"BECKMAN, SARAH","CARTER, SHARON II","SANADA, KONO","BEERE, JASON","SEVILLE, STACY",SWEEPZWEAK,DE LA COURTE
4,"FORTUNE, DOMINIC","ROM, SPACEKNIGHT","PARKER, MAY",MACHETE/FERDINAND LO,MINERVA [KREE],MONTANA,OX III,"DUGAN, TIMOTHY ALOYI",MAKKARI/MIKE KHARY/I,THIN MAN/BRUCE DICKS,...,TUSK,KID COLT II/ELRIC FR,MAXXAM,"CASSADA, MICHAEL","THOMAS, KIM HO TWAE",AMAZO-MAXI-WOMAN/,"REANDEAU, DR.",FANDRAL | MUTANT X-V,NECRODAMUS,BETA RAY BILL | MUTA
5,IRON MAN/TONY STARK,BLOB/FRED J. DUKES,JOCASTA,ZARAN/MAXIMILLIAN ZA,SUPREME INTELLIGENCE,"EVANS, DORIS",CORRUPTOR/JACKSON DA,JUSUR,"ZINOVIEV, IVAN",PATRIOT/JEFF MACE,...,TRITON DOPPELGANGER,AZIR,VENUS/APHRODITE/VICT,TENPIN/ALVIN HEALEY,TRAPSTER II/LARRY CU,"DEVEREAUX, IRIS",MECHAMAGE/,"VIENNEAU, JACQUES",CURSED LOTUS,None
6,"RAVEN, SABBATH II/EL",TORPEDO III/BROCK JO,GARGOYLE II/ISAAC CH,SILVERMANE/SILVIO MA,PRINCESS PYTHON/ZELD,CERISE II [SHI'AR],MIDGARD SERPENT,WHIRLWIND/DAVID CANN,PERRIKUS,GROTTU,...,GHOST GIRL/,SPOILSPORT/,PSI-WOLF/NIKOLAI VON,"BRADDOCK, JAMES","CAMPBELL, VALERIE","LUMPKIN, WILLIE | TI",WARFIST,None,None,None
7,CARNIVORE/COUNT ANDR,HYBRID/JAMES JIMMY M,COBRA/KLAUS VORHEES,SCARECROW II,DR. NEMESIS/DR. STOC,OJANI,BUZZ,"STARK, HOWARD","ROSS, COURTNEY","GAMBONNO, LUIGI",...,"DEVERAUX, INSPECTOR",MALPRACTICE,BENGAL/,CACTUS,"REYES, DR. DANIEL","SCICLUNA, MICHELE",KILLER CLOWN,None,None,None
8,GHOST,"KILLBURN, MACK",MALICE IV,PENANCE/MONET ST. CR,"LEWIS, DEVON","MCNAIR, TAD",IGOR,PLUTO [OLYMPIAN],CONDOR,BLACKBODY,...,ARIA,"FAWN, FELICIA","FORD, GERALD",SHADOW QUEEN/SHIALMA,"CROWLEY, SEN.","KANE, SUGAR",None,None,None,None
9,FU MANCHU,"CLARK, JOHN","RAVEN, LT. JACOB",UNUS THE UNTOUCHABLE,THUNDERBIRD II/JAMES,KAMIKAZE,TUVO,"REYES, DR. CECELIA","TAKACHIHO, HIRO",QUICKSAND,...,None,None,None,None,None,None,None,None,None,None


In [110]:
# compress graph into supernodes for better visualization

hero_compressed = [coms[x][0] for x in range(len(coms.transpose()))]

In [119]:
comp_graph = hero_graph
comp_graph.remove_nodes_from([item for item in hero_compressed if item not in heroes_sorted])

In [122]:
nx.draw_spectral(hero_graph, with_labels = True)

KeyboardInterrupt: 